In [19]:
import tensorflow as tf
print(tf.__version__)

2.1.1


In [20]:
# Clone the tensorflow models repository if it doesn't already exist

import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2191, done.
remote: Counting objects: 100% (2191/2191), done.
remote: Compressing objects: 100% (1895/1895), done.
remote: Total 2191 (delta 526), reused 932 (delta 270), pack-reused 0
Receiving objects: 100% (2191/2191), 30.41 MiB | 10.26 MiB/s, done.
Resolving deltas: 100% (526/526), done.


In [21]:
# Install the Object Detection API
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --user .

/tmp/mscoco/models/research
Processing /tmp/mscoco/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1578115 sha256=dcdaa466770b5dc80522f0a3b233c8cee06afc241f534b9acd19b50ee3378001
  Stored in directory: /tmp/pip-ephem-wheel-cache-65zopyc3/wheels/84/64/b5/79c1dc33b5fcbbfbb09c9d931a1bbca171c98d011887888f58
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
# Download and prepare coco dataset
%cd ~/models/research/object_detection/dataset_tools
!bash download_and_preprocess_mscoco.sh ~/data/mscoco

/home/jupyter/models/research/object_detection/dataset_tools
--2020-09-22 10:32:23--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.129.35
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.129.35|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  16.9MB/s    in 14m 8s  

2020-09-22 10:46:31 (21.8 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

Unzipping train2017.zip


In [24]:
PIPELINE_CONFIG_PATH="configs/tf2/ssd_mobilenet_v2_fpnlite_96x96_coco17_tpu-8.config"
MODEL_DIR="models/research/object_detection/models/ssd_mobilenet_v2_fpn"
NUM_TRAIN_STEPS=10000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1

In [30]:
# Run training
%cd ~/models/research/object_detection/
!python model_main_tf2.py \
  --model_dir=$MODEL_DIR --num_train_steps=$NUM_TRAIN_STEPS \
  --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES \
  --pipeline_config_path=$PIPELINE_CONFIG_PATH \
  --alsologtostderr

/home/jupyter/models/research/object_detection
2020-09-22 09:47:32.475201: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2020-09-22 09:47:32.475250: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-09-22 09:47:34.578752: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-22 09:47:35.202774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-22 09:47:35.203369: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with pro